<a href="https://colab.research.google.com/github/MindsMend-org/Biofeedback-Mindsync/blob/main/Project_EL_xdotcom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Model infer-Images Will be Generated For UK Lottery 7 Days before Event.
Wed & Sat

Check for Updates here  
x.com/@M0000000000004

Project EL Predicting The Future or Just reading reality's script.

Converted from nda.py located https://github.com/MindsMend-org/P_Project_EL


[HBR] binary representaion, wave graph and interpreter have been removed.

In [8]:
# Import necessary libraries
import shutil
import re
import os
import cv2  # pip install opencv-python
import numpy as np
import json
import csv
import time
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, concatenate, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from keras.models import model_from_json
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, Callback
from keras import metrics
from tensorflow.keras import backend as K
import sys
import requests

In [9]:
# Install necessary packages
!pip install opencv-python tensorflow keras

In [10]:
# Function to download the model from Google Drive https://drive.google.com/file/d/1z2QobGFmejh8aftgSOK3opS68mwZGAny/view?usp=sharing
# Function to download the model from Google Drive
def download_model():
    import gdown
    # The shared link is transformed into the file id for gdown to use
    model_file_id = '1z2QobGFmejh8aftgSOK3opS68mwZGAny'
    model_url = f'https://drive.google.com/uc?id={model_file_id}'
    model_output = '/content/STUAX_model.zip'
    gdown.download(model_url, model_output, quiet=False)

    # Check if the file is actually a rar file
    file_type = 'application/x-rar-compressed'  # mimetypes.guess_type(model_output)[0]
    if file_type == 'application/x-rar-compressed':
        # Create target directory if it doesn't exist
        target_dir = '/content/models/STUAX/'
        os.makedirs(target_dir, exist_ok=True)

        # Unrar the model file
        !unrar x /content/STUAX_model.zip /content/models/STUAX/
    else:
        print("The downloaded file is not a rar file. It might be the model file directly.")
        !mv /content/STUAX_model.rar /content/models/STUAX/model_weights.h5

        # Print the directory structure to verify
        !ls -R /content/models/STUAX/

        # Create target directory if it doesn't exist
        target_dir = '/content/models/STUAX/'
        os.makedirs(target_dir, exist_ok=True)

        # Unzip the model file
        !unzip -o /content/STUAX_model.zip -d /content/models/STUAX/

    # Print the directory structure to verify
    !ls -R /content/models/STUAX/

In [11]:
download_model()

Downloading...
From (original): https://drive.google.com/uc?id=1z2QobGFmejh8aftgSOK3opS68mwZGAny
From (redirected): https://drive.google.com/uc?id=1z2QobGFmejh8aftgSOK3opS68mwZGAny&confirm=t&uuid=558a3116-941c-49f0-be6a-69ba07f49851
To: /content/STUAX_model.zip
100%|██████████| 334M/334M [00:05<00:00, 55.8MB/s]



UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/STUAX_model.zip

Creating    /content/models/STUAX/STUAX                               OK
Extracting  /content/models/STUAX/STUAX/model_architecture.json            0%  OK 
Extracting  /content/models/STUAX/STUAX/model_weights.h5                   1%  2%  3%  5%  6%  7%  8% 10% 11% 12% 13% 15% 16% 17% 18% 20% 21% 22% 23% 25% 26% 27% 28% 30% 31% 32% 33% 35% 36% 37% 38% 40% 41% 42% 43% 45% 46% 47% 48% 50% 51% 52% 53% 55% 56% 57% 58% 60% 61% 62% 64% 65% 66% 67% 69% 70% 71% 72% 74% 75% 76% 77% 79% 80% 81% 82% 84% 85% 86% 87% 89% 90% 91% 92% 94% 95% 96% 97% 99%  OK 
A

In [12]:
# Function to download the cleaned results JSON from Google Drive
def download_cleaned_results():
    import gdown
    # The shared link is transformed into the file id for gdown to use
    results_file_id = '1PtAlKy0-IuEJds8rD51-rO0IYSGZlI6z'
    results_url = f'https://drive.google.com/uc?id={results_file_id}'
    results_output = '/content/cleaned_lottery_results.json'
    gdown.download(results_url, results_output, quiet=False)

In [13]:
# Add scraper get any result tbc # scrape_lottery_results.py
def get_lottery_results(date_str):
    api_key = "your_api_key"  # Replace with your actual API key
    url = f"https://api.example.com/lottery/results?date={date_str}&apiKey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

In [14]:
def create_dual_input_model(input_shape1, input_shape2, learning_rate=0.001, name='Default', debug=False):
    if debug:
        print(f'Creating Dual Input Model, learning rate: {learning_rate}')

    # First input
    input1 = Input(shape=input_shape1)
    x1 = Conv2D(16, (3, 3), activation='relu')(input1)
    x1 = MaxPooling2D((2, 2))(x1)
    x1 = Conv2D(32, (3, 3), activation='relu')(x1)
    x1 = MaxPooling2D((2, 2))(x1)
    x1 = Conv2D(64, (3, 3), activation='relu')(x1)
    x1 = Flatten()(x1)

    # Second input
    input2 = Input(shape=input_shape2)
    x2 = Conv2D(16, (3, 3), activation='relu')(input2)
    x2 = MaxPooling2D((2, 2))(x2)
    x2 = Conv2D(32, (3, 3), activation='relu')(x2)
    x2 = MaxPooling2D((2, 2))(x2)
    x2 = Conv2D(64, (3, 3), activation='relu')(x2)
    x2 = Flatten()(x2)

    # Merge both inputs
    merged = concatenate([x1, x2])

    # Fully connected layers
    x = Dense(128, activation='relu')(merged)
    x = Dense(64, activation='relu')(x)
    outputs = Dense(60, activation='sigmoid')(x)

    model = Model(inputs=[input1, input2], outputs=outputs, name=name)
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy',
                  metrics=['accuracy', metrics.Precision(), metrics.Recall()])

    if debug:
        model.summary()
    return model

In [15]:
def load_temporal_ai(model, name, best=True):
    model_name = model.name if name is None else name
    model_file = f'/content/models/STUAX/{model_name}/model_weights.h5'

    if best and os.path.exists(f'/content/models/STUAX/{model_name}/best_model.h5'):
        model_file = f'/content/models/{model_name}/best_model.h5'

    print(f'Loading {model_file}')
    model.load_weights(model_file)
    return model

In [16]:
def load_prediction_images(image_name, directory='/content/predict/'):
    img_path = os.path.join(directory, f"{image_name}.png")
    img_input2_path = os.path.join(directory, f"{image_name}_input2.png")

    img1 = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img1 = cv2.resize(img1, (60, 2000))
    img1 = np.expand_dims(img1, axis=-1) / 255.0

    img2 = cv2.imread(img_input2_path, cv2.IMREAD_GRAYSCALE)
    img2 = cv2.resize(img2, (60, 2000))
    img2 = np.expand_dims(img2, axis=-1) / 255.0

    return np.expand_dims(img1, axis=0), np.expand_dims(img2, axis=0)


In [17]:
def predict_with_threshold(model, x_input1, x_input2, threshold=0.5, predict_by_count_value=7, model_mode="NRF", debug=True):
    print(f'THRESHOLD HAS BEEN SET TO: {threshold}')
    if threshold == 0.0:
        print(f'Predicting top {predict_by_count_value} values')

    y_pred = model.predict([x_input1, x_input2])

    if threshold == 0.0:
        sorted_indices = np.argsort(y_pred[0])[::-1][:predict_by_count_value]
        top_percentages = y_pred[0][sorted_indices]
        find_threshold = min(top_percentages)
        y_pred_thresholded = (y_pred >= find_threshold).astype(int)
    else:
        y_pred_thresholded = (y_pred >= threshold).astype(int)

    return y_pred_thresholded, y_pred

In [18]:
def predict(model, image_name='x_val', threshold=0.6, model_mode="NRF", build_interpreter=False, debug=False, predict_count=9):
    if debug:
        print(f'Debug Mode: {debug} Model_Mode: {model_mode} Predict Count: {predict_count}')

    x_val1, x_val2 = load_prediction_images(image_name, directory='/content/predict/')
    y_pred_thresholded, y_pred_pcent = predict_with_threshold(model, x_val1, x_val2, threshold, model_mode=model_mode, predict_by_count_value=predict_count)

    if build_interpreter and debug:
        print(f'Building interpreter data...')

    return y_pred_thresholded, y_pred_pcent

In [19]:
def send_to_predict(model, from_path, dest_path='/content/predict', model_mode="NRF", threshold=0.5, predict_count=9, answers=[], graph=False, plot_alwayz=False, read_mode=False, build_interpreter=False, display_steps=False, interpreter_force_new_file=True, debug=False, show_at_end=False, nda_build_data=False):
    if debug:
        print(f'Sending To Predict.')

    y_pred_thresholded, y_pred_pcent = predict(model, 'x_val', threshold, model_mode=model_mode, predict_count=predict_count, build_interpreter=build_interpreter)

    if nda_build_data:
        print('NDA Parties Data Builder.')
        nda_dest_dir = "/content/NDA_parties/STUAX/images"
        pattern = r"/([a-zA-Z]+)-(\d{2}-\d{2}-\d{4})/(\d{4}-\d{2}-\d{2}T\d{2}-\d{2}-\d{2})"
        match = re.search(pattern, from_path)

        if match:
            day_abbreviation, date_value, timestamp_value = match.groups()
            nda_data_for = f"{day_abbreviation}_{date_value}"
            nda_data_from = timestamp_value
            nda_stamp = f"FC_M_MADE[{nda_data_from}]_For_Target[{nda_data_for}]"
            directory_path = os.path.join(nda_dest_dir, nda_stamp)

            os.makedirs(directory_path, exist_ok=True)
            shutil.copy("/content/predict/x_val.png", os.path.join(directory_path, "x_val.png"))
            shutil.copy("/content/predict/x_val_input2.png", os.path.join(directory_path, "x_val_input2.png"))
            print("NDA Files copied.")

    if read_mode:
        return y_pred_thresholded, y_pred_pcent

    return y_pred_thresholded, y_pred_pcent

In [20]:
def print_thresholded_values(y_pred_thresholded, y_pred_pcent, answers, threshold, read_mode=False, display_steps=False, addto_interpreter=False, file="", debug=False, show_at_end=False):
    reset, red, white, yellow = "\033[0m", "\033[31m", "\033[37m", "\033[93m"
    print("LABEL:")
    for sublist in y_pred_thresholded:
        for j, val in enumerate(sublist):
            color = red if j in answers else yellow
            val = 1 if j in answers else 0
            print(f"{color}{val}{reset}", end=' ')
        print()
    print(reset, end='')

    matched, predicted = [], []
    if display_steps:
        print("y_pred_thresholded:")
    for sublist in y_pred_thresholded:
        for j, val in enumerate(sublist):
            color = red if j in answers else yellow
            if val > threshold:
                predicted.append(j)
                if color == red:
                    matched.append(j)
                print(f"{color}{val}{reset}", end=' ')
            else:
                print(f"{white}{val}{reset}", end=' ')
        print()
    print(reset, end='')

    print("y_pred_pcent:", end=' ')
    for sublist in y_pred_pcent:
        for j, val in enumerate(sublist):
            if val > threshold:
                color = red if j in answers else white
                print(f"{color}{val}{reset}", end=' ')
        print()
    print(reset, end='')

    print(f'Predicted: {predicted}')
    print(f"answers: {', '.join(f'{red}{val}{reset}' for val in answers)}")
    print(f'Matched: {matched}')
    print(f"{white}{reset}")

In [21]:
def init_ai(name='Default', learning_rate=0.001):
    num_channels = 1
    height, width = 2000, 60
    input_shape1 = (height, width, num_channels)
    input_shape2 = (height, width, num_channels)
    model = create_dual_input_model(input_shape1, input_shape2, learning_rate, name=name)
    return model

In [22]:
def get_prediction_list_to_process(base_directory="/content/images"):
    if os.path.exists(base_directory) and os.path.isdir(base_directory):
        subdirectories = [name for name in os.listdir(base_directory) if os.path.isdir(os.path.join(base_directory, name))]
        print("Subdirectories in 'images':", subdirectories)
        return subdirectories
    print("The 'images' directory does not exist.")
    return []

In [45]:
def copy_from_to(dir_from="", dir_to="/content/predict"):
    # Ensure the destination directory exists
    os.makedirs(dir_to, exist_ok=True)

    # Construct the full source directory path
    dir_from = os.path.join("/content/images", dir_from)

    # Copy the files from the source to the destination directory
    shutil.copy(os.path.join(dir_from, "x_val.png"), os.path.join(dir_to, "x_val.png"))
    shutil.copy(os.path.join(dir_from, "x_val_input2.png"), os.path.join(dir_to, "x_val_input2.png"))

    print("Files copied.")

In [24]:
def load_cleaned_results(file_path='/content/cleaned_lottery_results.json'):
    with open(file_path, 'r') as file:
        return json.load(file)

In [25]:
def get_labels_for_date(date, cleaned_results):
    return cleaned_results.get(date, [])

In [26]:
# Constants
# with threshold set to 0.0 then it switches to predict a count specified by user, You here its set to 4 predicted numbers, The UK lottery has 7 numbers 6 main 1 bonus reality-wise its 7
PREDICTION_COUNT = 4  # how many to predict bearing in mind this model is trained 150k of min 2,658,391,066 for 1 per possibility but  (2,658,391,066 *10k) id like to see this done. Ultimate goal is[ABSOLUTE]
threshold = 0.0  # set to 0.0 for count mode, you can set to 0.5 would be %50
SHOW_AT_END = False
build_interpreter = False
debug = False
DisplaySteps = False
interpreter_file_dir = ""

In [27]:
# Download the model
#download_model()

In [28]:
# Download the UK lottery results
# Download the cleaned lottery results
download_cleaned_results()

Downloading...
From: https://drive.google.com/uc?id=1PtAlKy0-IuEJds8rD51-rO0IYSGZlI6z
To: /content/cleaned_lottery_results.json
100%|██████████| 7.56k/7.56k [00:00<00:00, 13.1MB/s]


In [29]:
# Load the cleaned lottery results
def load_cleaned_results(file_path='/content/cleaned_lottery_results.json'):
    with open(file_path, 'r') as file:
        return json.load(file)

lottery_results = load_cleaned_results()

In [30]:
# Initialize the model
model = init_ai('STUAX')
model = load_temporal_ai(model, name='STUAX', best=False)

Loading /content/models/STUAX/STUAX/model_weights.h5


In [31]:
# images link https://drive.google.com/file/d/12kGBk7aod9BdnzjD2pjiChVKIT-kst0G/view?usp=sharing
# permissions set to anyone with link to google is like cba/like2anoy

In [32]:
# Step 1: Install required libraries
!pip install gdown rarfile

# Step 2: Download the .rar file using gdown
import gdown

file_id = '12kGBk7aod9BdnzjD2pjiChVKIT-kst0G'
gdown.download(f'https://drive.google.com/uc?id={file_id}', 'images.rar', quiet=False)

# Step 3: Extract the .rar file
import rarfile
import os

with rarfile.RarFile('images.rar') as rf:
    rf.extractall('/content/images')

print("Extraction completed.")



Downloading...
From: https://drive.google.com/uc?id=12kGBk7aod9BdnzjD2pjiChVKIT-kst0G
To: /content/images.rar
100%|██████████| 381k/381k [00:00<00:00, 59.2MB/s]


Extraction completed.


In [35]:
# Get the list of directories to process
to_process = get_prediction_list_to_process("/content/images/images")

Subdirectories in 'images': ['imagesFC_M_MADE[2023-09-24T15-16-29]_For_Target[Sat_30-09-2023]', 'imagesFC_M_MADE[2023-11-04T17-00-16]_For_Target[Wed_08-11-2023]', 'imagesFC_M_MADE[2023-10-24T18-59-05]_For_Target[Wed_25-10-2023]', 'imagesFC_M_MADE[2023-09-29T18-34-28]_For_Target[Sat_30-09-2023]', 'imagesFC_M_MADE[2023-10-01T15-55-25]_For_Target[Wed_04-10-2023]', 'imagesFC_M_MADE[2023-11-17T12-29-13]_For_Target[Sat_18-11-2023]', 'imagesFC_M_MADE[2023-10-15T15-02-00]_For_Target[Wed_18-10-2023]', 'imagesFC_M_MADE[2023-11-30T21-55-41]_For_Target[Sat_02-12-2023]', 'imagesFC_M_MADE[2023-11-14T16-40-52]_For_Target[Wed_15-11-2023]', 'imagesFC_M_MADE[2023-11-28T20-47-28]_For_Target[Wed_29-11-2023]', 'imagesFC_M_MADE[2023-10-24T22-44-37]_For_Target[Wed_25-10-2023]', 'imagesFC_M_MADE[2023-09-24T15-43-16]_For_Target[Sat_30-09-2023]', 'imagesFC_M_MADE[2023-09-26T13-36-10]_For_Target[Wed_27-09-2023]', 'imagesFC_M_MADE[2023-10-06T22-34-23]_For_Target[Sat_07-10-2023]', 'imagesFC_M_MADE[2023-10-26T15-24

In [46]:
# It is what it is

for item in to_process:
    print(f'\nDissabled&Removed[Build Holographic Binary Representation] for file: {item}')


    # Regular expression pattern to extract made and for_target
    pattern = r"FC_M_MADE\[(.*?)\]_For_Target\[(.*?)\]"

    # Search for the pattern in the example string
    match = re.search(pattern, item)

    if match:
        made = match.group(1)
        for_target = match.group(2)
        print(f"made = '{made}'")
        print(f"for_target = '{for_target}'")
    else:
        print("Pattern not found")

    # Remove brackets and format target_label
    target_label = for_target.replace('[', '').replace(']', '').replace('_', '-')
    print('-')
    print(target_label)
    print('-')

    # Construct full path
    full_path = f'/content/images/images/{item}'
    if os.path.exists(full_path):
        copy_from_to(dir_from=full_path, dir_to="predict")
    else:
        print(f"Error: Directory {full_path} does not exist.")
        continue

    # Fetch labels from cleaned results
    labels = []
    for date_key, numbers in lottery_results.items():
        if target_label in date_key:
            labels = numbers
            break
    flattened_labels = [item for sublist in labels for item in sublist]

    print(flattened_labels)

    y_pred_thresholded, y_pred_pcent = predict(model=model, threshold=threshold, predict_count=PREDICTION_COUNT)
    print(y_pred_thresholded, y_pred_pcent)
    print_thresholded_values(y_pred_thresholded, y_pred_pcent, flattened_labels, threshold,
                             display_steps=DisplaySteps, addto_interpreter=build_interpreter,
                             file=interpreter_file_dir, debug=debug, show_at_end=SHOW_AT_END)


Dissabled&Removed[Build Holographic Binary Representation] for file: imagesFC_M_MADE[2023-09-24T15-16-29]_For_Target[Sat_30-09-2023]
made = '2023-09-24T15-16-29'
for_target = 'Sat_30-09-2023'
-
Sat-30-09-2023
-
Files copied.
[1, 8, 14, 20, 24, 40, 52]
THRESHOLD HAS BEEN SET TO: 0.0
Predicting top 4 values
1/1 [==============================] - 1s 623ms/step
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0]] [[0.0000000e+00 3.4739911e-05 4.1316654e-18 2.6465646e-12 3.6014469e-19
  1.3299235e-16 9.8550886e-01 1.7008224e-09 5.4366140e-24 1.7590531e-07
  6.5521143e-02 2.6079669e-04 1.0381992e-06 3.9482400e-01 3.1114016e-06
  2.2520097e-11 9.0478607e-06 6.8944706e-18 7.0639401e-09 1.5867051e-09
  1.4207491e-16 7.8454777e-21 1.2289106e-13 8.3669825e-03 5.2005163e-15
  2.1234497e-08 9.9999994e-01 8.4776072e-15 2.2881976e-04 3.5011133e-05
  3.9031130e-15 5.2817133e-09 6.3472009e-13 2.0671981e-08 1.2832064e-17
  1.65034

Part Of FoldingCircles
mince@foldingcircles.co.uk